In [200]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
# from sklearn.linear_model import Lasso
from sklearn.metrics import accuracy_score
import torch
from torch import nn

In [201]:
titds=pd.read_csv('datasets/TITANICtrain.csv')
titds

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [202]:
# preprocessing
titds=titds.drop(columns='Cabin', axis=1)
titds['Age'].fillna(titds['Age'].mean(),inplace=True)
titds['Embarked'].fillna('S',inplace=True)
Survivedmale=titds.loc[(titds['Sex'] =='male') & (titds['Survived'] == 1)]
NonSurvivedmale=titds.loc[(titds['Sex'] =='male') & (titds['Survived'] == 0)]
Survivedfemale=titds.loc[(titds['Sex'] =='female') & (titds['Survived'] == 1)]
NonSurvivedfemale=titds.loc[(titds['Sex'] =='female') & (titds['Survived'] == 0)]
replacement_mapping_dict = {
    "male": 0,
    "female":1
}
titds["Sex"].replace(replacement_mapping_dict, inplace=True)
replacement_mapping_dict2 = {
    "S": 0,
    "C":1,
    "Q":2
}
titds["Embarked"].replace(replacement_mapping_dict2, inplace=True)
k=[]
for i in titds["Name"]:
    k.append(i)
k
l=[]
for i in k:
    l.append(i.split(",")[1])
l
j=[]
for i in l:
    j.append(i.split(".")[0])
j
titds["titles"]=j
titds["titles"].value_counts()

 Mr              517
 Miss            182
 Mrs             125
 Master           40
 Dr                7
 Rev               6
 Mlle              2
 Major             2
 Col               2
 Ms                1
 Don               1
 Sir               1
 the Countess      1
 Lady              1
 Jonkheer          1
 Mme               1
 Capt              1
Name: titles, dtype: int64

In [203]:
X

tensor([[ 3.0000,  0.0000, 22.0000,  ...,  7.2500,  0.0000,  0.0000],
        [ 1.0000,  1.0000, 38.0000,  ..., 71.2833,  1.0000,  2.0000],
        [ 3.0000,  1.0000, 26.0000,  ...,  7.9250,  0.0000,  1.0000],
        ...,
        [ 3.0000,  1.0000, 29.6991,  ..., 23.4500,  0.0000,  1.0000],
        [ 1.0000,  0.0000, 26.0000,  ..., 30.0000,  1.0000,  0.0000],
        [ 3.0000,  0.0000, 32.0000,  ...,  7.7500,  2.0000,  0.0000]])

In [204]:
y

tensor([0., 1., 1., 1., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 1., 0., 1.,
        0., 1., 0., 1., 1., 1., 0., 1., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0.,
        1., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1.,
        0., 1., 1., 0., 1., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0.,
        0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 0., 1., 1., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1.,
        0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1.,
        0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0.,
        0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1.,
        0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 1., 1., 0., 1., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0., 0.,
        1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 1., 0., 1., 0., 0., 0., 1.,
        1., 0., 1., 0., 1., 0., 0., 0., 

In [205]:
X= torch.from_numpy(np.asarray(X)).type(torch.float)
y= torch.from_numpy(np.asarray(y)).type(torch.float)

In [206]:
type(X),type(y)

(torch.Tensor, torch.Tensor)

In [207]:
#train test split
X_train, X_test, y_train, y_test= train_test_split(X,y,test_size=0.2,random_state=42)
                                                                      

In [208]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

(torch.Size([712, 8]),
 torch.Size([179, 8]),
 torch.Size([712]),
 torch.Size([179]))

In [209]:
#builing a model
#subclas nn.Module
class TitanicModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer_1=nn.Linear(in_features=8,out_features=8)
        self.layer_2=nn.Linear(in_features=8,out_features=8)
        self.layer_3=nn.Linear(in_features=8,out_features=1)
        
        self.relu=nn.ReLU()
    
    def forward(self,x):
        return self.layer_3(self.relu(self.layer_2(self.relu(self.layer_1(x)))))
    
    
model_A=TitanicModel()
model_A

TitanicModel(
  (layer_1): Linear(in_features=8, out_features=8, bias=True)
  (layer_2): Linear(in_features=8, out_features=8, bias=True)
  (layer_3): Linear(in_features=8, out_features=1, bias=True)
  (relu): ReLU()
)

In [210]:
#accuracy function
#calc accuracy
def accuracy_fn(y_true,y_pred):
    correct=torch.eq(y_true,y_pred).sum().item()
    acc=(correct/len(y_pred)) * 100
    return acc

In [211]:
#view first 5 outputs of forward pass on test data
model_A.eval()
with torch.inference_mode():
    y_logits=model_A(X_test)[:5]
y_logits

tensor([[0.2269],
        [0.0981],
        [0.0869],
        [0.0864],
        [0.1373]])

In [212]:
#use sigmoid activation funtion on our model logits to tun them into prediction probablities
y_pred_probs = torch.sigmoid(y_logits)
y_pred_probs

tensor([[0.5565],
        [0.5245],
        [0.5217],
        [0.5216],
        [0.5343]])

In [213]:
torch.round(y_pred_probs)

tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.]])

In [214]:
#find the predicted lables
y_preds = torch.round(y_pred_probs)

#in full
y_pred_labels=torch.round(torch.sigmoid(model_A(X_test)))[:5]

# check for equality
print(torch.eq(y_preds.squeeze(),y_pred_labels.squeeze()))

#get rid of extra dimension
y_preds.squeeze()

tensor([True, True, True, True, True])


tensor([1., 1., 1., 1., 1.])

In [215]:
#loss function
loss_fn= nn.BCEWithLogitsLoss()
#optimizer
optimizer=torch.optim.Adam(model_A.parameters(),
                          lr=0.01)

In [216]:
#training loop
torch.manual_seed(42)
epochs=10000
for epoch in range(epochs):
    model_A.train()
    y_logits=model_A(X_train).squeeze()
    y_pred= torch.round(torch.sigmoid(y_logits))
    
    loss=loss_fn(y_logits,y_train)
    acc= accuracy_fn(y_true=y_train,y_pred=y_pred)
    
    optimizer.zero_grad()
    
    loss.backward()
    
    optimizer.step()
    
    model_A.eval()
    with torch.inference_mode():
        test_logits = model_A(X_test).squeeze()
        test_pred = torch.round(torch.sigmoid(test_logits))
        
        test_loss= loss_fn(test_logits,y_test)
        test_acc= accuracy_fn(y_true=y_test,y_pred=test_pred)
        
    if epoch%1000==0:
        print(f"Epoch: {epoch} | loss: {loss: .5f}, Acc:{acc:.2f}% | Test loss: {test_loss:.5f}, Test acc: {test_acc:.2f}%")

Epoch: 0 | loss:  0.71216, Acc:36.94% | Test loss: 0.68174, Test acc: 46.37%
Epoch: 1000 | loss:  0.34298, Acc:86.80% | Test loss: 0.49285, Test acc: 79.33%
Epoch: 2000 | loss:  0.33549, Acc:86.52% | Test loss: 0.48723, Test acc: 80.45%
Epoch: 3000 | loss:  0.33400, Acc:86.80% | Test loss: 0.52211, Test acc: 78.77%
Epoch: 4000 | loss:  0.33476, Acc:86.10% | Test loss: 0.55544, Test acc: 79.33%
Epoch: 5000 | loss:  0.34062, Acc:86.52% | Test loss: 0.50971, Test acc: 81.56%
Epoch: 6000 | loss:  0.33094, Acc:86.94% | Test loss: 2.07614, Test acc: 78.21%
Epoch: 7000 | loss:  0.32946, Acc:86.38% | Test loss: 0.61733, Test acc: 80.45%
Epoch: 8000 | loss:  0.32419, Acc:87.22% | Test loss: 0.49151, Test acc: 78.21%
Epoch: 9000 | loss:  0.32503, Acc:86.80% | Test loss: 0.55407, Test acc: 78.21%


In [217]:
testds=pd.read_csv('datasets/TITANICtest.csv')
testds

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [218]:
#preprocessing
testds=testds.drop(columns='Cabin', axis=1)
testds['Age'].fillna(testds['Age'].mean(),inplace=True)
testds['Fare'].fillna(testds['Fare'].mean(),inplace=True)
replacement_mapping_dict = {
    "male": 0,
    "female":1
}
testds["Sex"].replace(replacement_mapping_dict, inplace=True)
replacement_mapping_dict2 = {
    "S": 0,
    "C":1,
    "Q":2
}
testds["Embarked"].replace(replacement_mapping_dict2, inplace=True)
k=[]
for i in testds["Name"]:
    k.append(i)
k
l=[]
for i in k:
    l.append(i.split(",")[1])
l
j=[]
for i in l:
    j.append(i.split(".")[0])
j
testds["titles"]=j
replacement_mapping_dict3 = {
    " Mr": 0,
    " Miss":1,
    " Mrs":2,
    " Master":3,
    " Dr":4,
    " Rev":5,
    " Major":6,
    " Col":7,
    " Mlle":8,
    " Lady":9,
    " Ms":10,
    " Dona":11,
    " Sir":12,
    " Jonkheer":13,
    " Capt":14,
    " Mme":15,
    " the Countess":16,
       
}
testds["titles"].replace(replacement_mapping_dict3, inplace=True)


In [219]:
testds

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,titles
0,892,3,"Kelly, Mr. James",0,34.50000,0,0,330911,7.8292,2,0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",1,47.00000,1,0,363272,7.0000,0,2
2,894,2,"Myles, Mr. Thomas Francis",0,62.00000,0,0,240276,9.6875,2,0
3,895,3,"Wirz, Mr. Albert",0,27.00000,0,0,315154,8.6625,0,0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",1,22.00000,1,1,3101298,12.2875,0,2
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",0,30.27259,0,0,A.5. 3236,8.0500,0,0
414,1306,1,"Oliva y Ocana, Dona. Fermina",1,39.00000,0,0,PC 17758,108.9000,1,11
415,1307,3,"Saether, Mr. Simon Sivertsen",0,38.50000,0,0,SOTON/O.Q. 3101262,7.2500,0,0
416,1308,3,"Ware, Mr. Frederick",0,30.27259,0,0,359309,8.0500,0,0


In [220]:
#removing name,ticket pid
X_test=testds.drop(columns=['PassengerId','Name','Ticket'],axis=1)
# X_test['titles'].value_counts()


In [221]:
X_test

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,titles
0,3,0,34.50000,0,0,7.8292,2,0
1,3,1,47.00000,1,0,7.0000,0,2
2,2,0,62.00000,0,0,9.6875,2,0
3,3,0,27.00000,0,0,8.6625,0,0
4,3,1,22.00000,1,1,12.2875,0,2
...,...,...,...,...,...,...,...,...
413,3,0,30.27259,0,0,8.0500,0,0
414,1,1,39.00000,0,0,108.9000,1,11
415,3,0,38.50000,0,0,7.2500,0,0
416,3,0,30.27259,0,0,8.0500,0,0
